In [ ]:
!pip uninstall -y pandas pandasai
!pip install pandas==2.2.2
!pip install pandasai==2.2.5 --no-deps


In [ ]:
!rm -rf /content/house-price-ml-pipeline
!git clone https://github.com/kathyanusha05465/house-price-ml-pipeline.git /content/house-price-ml-pipeline
%cd /content/house-price-ml-pipeline


In [ ]:
# ✅ Portable setup for Colab or local
import os, sys
from pathlib import Path
import pandas as pd

IN_COLAB = "google.colab" in sys.modules or "COLAB_RELEASE_TAG" in os.environ

if IN_COLAB:
    ROOT = Path("/content/house-price-ml-pipeline")
else:
    ROOT = Path.cwd().parents[0] if Path.cwd().name == "Notebooks" else Path.cwd()

DATA_DIR = ROOT / "Data"
REPORTS_DIR = ROOT / "Reports"
ARTIFACTS_DIR = ROOT / "artifacts"
REPORTS_DIR.mkdir(parents=True, exist_ok=True)
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)

print("📁 CWD:", Path.cwd())
print("📂 ROOT:", ROOT)
print("✅ CSV Found:", (DATA_DIR / "real_estate.csv").exists())


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    r2_score
)


Load and preview dataset

In [ ]:
houseprice_df = pd.read_csv(DATA_DIR / "real_estate.csv")
display(houseprice_df.head())
print("Dataset shape:", houseprice_df.shape)


Load API Key & Initialize PandasAI

In [ ]:
# 🔐 Read OpenAI key securely from Colab Secrets (sidebar → 🔑)
from google.colab import userdata

api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("⚠️ Please add your OPENAI_API_KEY in Colab's Secrets sidebar (🔑).")

from pandasai import SmartDataframe
from pandasai.llm import OpenAI

llm = OpenAI(api_token=api_key)
houseprice = SmartDataframe(houseprice_df, config={"llm": llm})
house_price = houseprice  # alias


**Data Cleaning and Preparation**

In this section, we clean and prepare the dataset by renaming columns, removing extra spaces, and converting column names to a consistent format. This ensures that both PandasAI and sklearn models can access features correctly.

In [ ]:
# --- Clean & Standardize Column Names ---
houseprice_df.columns = (
    houseprice_df.columns
        .str.strip().str.lower().str.replace(" ", "_", regex=False)
)
houseprice_df.rename(columns={"no": "transaction_id"}, inplace=True)

display(houseprice_df.head())
houseprice_df.info()

# Recreate SmartDataframe after renaming
houseprice = SmartDataframe(houseprice_df, config={"llm": llm})
house_price = houseprice


**Missing Values & Duplicates**

Next, we check for any missing or duplicate rows in the dataset to ensure data integrity before modeling.

In [ ]:
print("\nMissing values per column:")
print(houseprice_df.isna().sum())

print("\nDuplicate rows:", houseprice_df.duplicated().sum())


**Data Distribution and Outliers**

Outliers can distort model performance. We cap extreme values at the 97th percentile and remove a few high-value outliers in the target variable.

In [ ]:
# --- Outlier Handling ---
p97_mrt = houseprice_df["x3_distance_to_the_nearest_mrt_station"].quantile(0.97)
houseprice_df["x3_distance_to_the_nearest_mrt_station"] = (
    houseprice_df["x3_distance_to_the_nearest_mrt_station"].clip(upper=p97_mrt)
)

# Drop top 3 target outliers
houseprice_df = houseprice_df.sort_values("y_house_price_of_unit_area").iloc[:-3].reset_index(drop=True)

print("✅ Outliers handled. Remaining rows:", len(houseprice_df))

# Recreate SmartDataframe after modifying data
houseprice = SmartDataframe(houseprice_df, config={"llm": llm})
house_price = houseprice

**Exploratory Data Analysis**

Below is a quick summary and correlation heatmap to understand relationships between features.

In [ ]:
display(houseprice_df.describe())

plt.figure(figsize=(8,6))
sns.heatmap(houseprice_df.corr(numeric_only=True), annot=False, cmap="coolwarm")
plt.title("Correlation Heatmap")
plt.show()


**Model Preparation & Training (Decision Tree)**

We define features, split data into training and validation sets, and use a Decision Tree Regressor with GridSearchCV to tune parameters and evaluate model performance.

In [ ]:
FEATURES = [
    "x3_distance_to_the_nearest_mrt_station",
    "x4_number_of_convenience_stores",
    "x5_latitude",
    "x6_longitude",
]
TARGET = "y_house_price_of_unit_area"

X = houseprice_df[FEATURES]
y = houseprice_df[TARGET]

print("✅ Features and target ready.")


Train/Test Split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.40, random_state=42
)

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")


Decision Tree Regression with GridSearchCV

In [ ]:
param_grid = {
    "max_depth": [2, 3, 4, 5, 6, 8, 10],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
}

grid = GridSearchCV(
    DecisionTreeRegressor(random_state=42),
    param_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

grid.fit(X_train, y_train)
best_dt = grid.best_estimator_
print("Best Parameters:", grid.best_params_)

pred = best_dt.predict(X_val)

rmse = mean_squared_error(y_val, pred, squared=False)
mae  = mean_absolute_error(y_val, pred)
mape = mean_absolute_percentage_error(y_val, pred) * 100
r2   = r2_score(y_val, pred)

print(f"\n✅ Decision Tree Evaluation:")
print(f"RMSE: {rmse:.4f} | MAE: {mae:.4f} | MAPE: {mape:.2f}% | R²: {r2:.4f}")


**Model Evaluation Results**

The table below summarizes the performance of all models from the project. Random Forest achieved the best performance, but the Decision Tree model (this notebook) provides interpretable results with minimal complexity.

In [ ]:
results = pd.DataFrame([
    ["Linear Regression", 1.2256, 7.6140, 5.8800, -1.4676, 16.5966],
    ["Random Forest", 0.2008, 5.7360, 4.4204, -2.8545, 12.9027],
    ["KNN", 0.7322, 6.9516, 5.5362, -3.0110, 16.2728],
    ["Decision Tree", -3.7591, 7.5989, 5.6497, -10.0000, 10.0000],
],
columns=["Model", "ME ($)", "RMSE ($)", "MAE ($)", "MPE (%)", "MAPE (%)"])

display(results)
